In [17]:
import csv
import lzstring
from collections import namedtuple, Counter
import json
from memoize import memoize
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import sys
import json

In [37]:
with open('data/intervention_polling.json') as file:
    data = json.load(file)

In [38]:
print (data)

{'just_right': [None], 'reddit/show_timer_banner': [None, 'just_right', 'just_right', 'just_right'], 'generic/show_timer_banner': ['just_right', 'just_right', 'just_right', 'just_right', 'just_right', 'just_right', 'just_right', 'not_intense', 'not_intense', 'just_right', 'just_right', 'not_intense', 'just_right', 'just_right', 'just_right', 'just_right', 'just_right', 'not_intense', 'too_intense', 'not_intense', 'just_right', 'just_right', 'just_right', 'too_intense', 'just_right', 'too_intense', 'just_right', 'just_right', 'just_right', 'too_intense', 'just_right', 'just_right', 'not_intense', 'just_right', 'just_right', 'just_right', 'just_right', 'just_right', 'just_right', 'just_right', 'not_intense', 'just_right', 'not_intense', 'too_intense', 'just_right', 'just_right', 'not_intense', 'just_right', 'not_intense', 'just_right', 'just_right', 'just_right', 'not_intense', 'just_right', 'too_intense', 'just_right', 'just_right', 'not_intense', 'just_right', 'just_right', 'just_right

In [39]:
res = {}
for i in data.keys():
    s = i
    ind = s.find('/')
    if ind == -1:
        continue
    s = s[ind + 1: ]
    if s not in res.keys():
        res[s] = []
    for j in data[i]:
        if j == 'just_right':
            res[s].append(2)
        if j == 'not_intense':
            res[s].append(1)
        if j == 'too_intense':
            res[s].append(3)

print (res)

{'show_timer_banner': [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 3, 1, 2, 2, 2, 3, 2, 3, 2, 2, 2, 3, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 3, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 3, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 2, 2, 3, 2, 2, 2, 1, 1, 3, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2], 'toast_notifications': [1, 1, 1, 2, 3, 2, 2, 3, 2, 1, 2, 2, 1, 1, 2, 3, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 3, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 2, 2, 2, 2, 3, 2, 3, 1, 1, 3, 1, 2, 2, 3, 2, 2, 2, 1, 1, 3, 1, 1, 2, 2, 1, 1, 3, 2, 2, 2, 2, 1], 'feed_injection_timer': [2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 3, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2], 'remove_comments': [2, 2, 3, 2, 2, 2, 3, 2, 2, 1, 1, 2, 1, 3, 2], 'prompt_before_watch': [2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 3, 2, 2, 2, 1, 2

In [40]:
for k in res.keys():
    res[k] = np.sum(res[k]) / len(res[k])
print (res)

{'show_timer_banner': 1.8691588785046729, 'toast_notifications': 1.8556701030927836, 'feed_injection_timer': 1.8205128205128205, 'remove_comments': 2.0, 'prompt_before_watch': 1.894736842105263, 'remove_sidebar_links': 1.880952380952381, 'show_user_info_interstitial': 1.8928571428571428, 'close_tab_timer': 2.1724137931034484, 'remove_clickbait': 1.8717948717948718, 'rich_notifications': 1.8181818181818181, 'remove_comment_section': 1.8918918918918919, 'block_after_interval_per_visit': 1.90625, 'remove_news_feed': 2.1904761904761907, 'prompt_reason': 2.027027027027027, 'make_user_wait': 1.896551724137931, 'remove_twitter_feed': 2.0, 'scroll_blocker': 2.2, 'show_user_info_banner': 1.7857142857142858, 'remove_recommended': 2.1153846153846154, 'stop_autoplay': 2.0, 'infinite_alarm': 2.0, 'link_articles': 2.0}


In [41]:
tmp = {}
for x in res.keys():
    tmp[x] = res[x]
for k in res.keys():
    if abs(float(res[k]) - 2) < 0.5:
        res[k] = 'medium'
    elif res[k] < 2:
        res[k] = 'easy'
    else:
        res[k] = 'hard'
        
print (res)

{'show_timer_banner': 'medium', 'toast_notifications': 'medium', 'feed_injection_timer': 'medium', 'remove_comments': 'medium', 'prompt_before_watch': 'medium', 'remove_sidebar_links': 'medium', 'show_user_info_interstitial': 'medium', 'close_tab_timer': 'medium', 'remove_clickbait': 'medium', 'rich_notifications': 'medium', 'remove_comment_section': 'medium', 'block_after_interval_per_visit': 'medium', 'remove_news_feed': 'medium', 'prompt_reason': 'medium', 'make_user_wait': 'medium', 'remove_twitter_feed': 'medium', 'scroll_blocker': 'medium', 'show_user_info_banner': 'medium', 'remove_recommended': 'medium', 'stop_autoplay': 'medium', 'infinite_alarm': 'medium', 'link_articles': 'medium'}


In [42]:
print (tmp)

{'show_timer_banner': 1.8691588785046729, 'toast_notifications': 1.8556701030927836, 'feed_injection_timer': 1.8205128205128205, 'remove_comments': 2.0, 'prompt_before_watch': 1.894736842105263, 'remove_sidebar_links': 1.880952380952381, 'show_user_info_interstitial': 1.8928571428571428, 'close_tab_timer': 2.1724137931034484, 'remove_clickbait': 1.8717948717948718, 'rich_notifications': 1.8181818181818181, 'remove_comment_section': 1.8918918918918919, 'block_after_interval_per_visit': 1.90625, 'remove_news_feed': 2.1904761904761907, 'prompt_reason': 2.027027027027027, 'make_user_wait': 1.896551724137931, 'remove_twitter_feed': 2.0, 'scroll_blocker': 2.2, 'show_user_info_banner': 1.7857142857142858, 'remove_recommended': 2.1153846153846154, 'stop_autoplay': 2.0, 'infinite_alarm': 2.0, 'link_articles': 2.0}


In [43]:
interventions = []
for x in res.keys():
    interventions.append(x)
print (interventions)

['show_timer_banner', 'toast_notifications', 'feed_injection_timer', 'remove_comments', 'prompt_before_watch', 'remove_sidebar_links', 'show_user_info_interstitial', 'close_tab_timer', 'remove_clickbait', 'rich_notifications', 'remove_comment_section', 'block_after_interval_per_visit', 'remove_news_feed', 'prompt_reason', 'make_user_wait', 'remove_twitter_feed', 'scroll_blocker', 'show_user_info_banner', 'remove_recommended', 'stop_autoplay', 'infinite_alarm', 'link_articles']


In [45]:
interventions = sorted(interventions, key=lambda x: tmp[x])

In [47]:
print (interventions)

['show_user_info_banner', 'rich_notifications', 'feed_injection_timer', 'toast_notifications', 'show_timer_banner', 'remove_clickbait', 'remove_sidebar_links', 'remove_comment_section', 'show_user_info_interstitial', 'prompt_before_watch', 'make_user_wait', 'block_after_interval_per_visit', 'remove_comments', 'remove_twitter_feed', 'stop_autoplay', 'infinite_alarm', 'link_articles', 'prompt_reason', 'remove_recommended', 'close_tab_timer', 'remove_news_feed', 'scroll_blocker']
